In [3]:
%pip install openai
%pip install chess
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import required libraries
import os
import openai
import json
import numpy as np
import pandas as pd
import chess
import copy
import time
import datetime


from ast import literal_eval
from dotenv import load_dotenv, find_dotenv
from sklearn.model_selection import train_test_split

In [5]:
# create dataframe from downloaded dataset
df = pd.read_csv('data/db.csv')
# create dataframe of one move chess puzzles
df_one = df[df['Themes'].str.contains('mateIn1')].reset_index(drop=True)

In [6]:
# import environment variable into notebook
_ = load_dotenv(find_dotenv())  # read local .env file
# openai.api_key = ''
# list openai available models given the api key.
ids = [item["id"] for item in (openai.Model.list())['data']]
print(ids)

['gpt-4-0613', 'text-davinci-001', 'text-search-curie-query-001', 'davinci', 'text-babbage-001', 'curie-instruct-beta', 'text-davinci-003', 'gpt-3.5-turbo-16k-0613', 'davinci-similarity', 'code-davinci-edit-001', 'text-similarity-curie-001', 'text-embedding-ada-002', 'gpt-3.5-turbo-16k', 'ada-code-search-text', 'text-search-ada-query-001', 'gpt-4-0314', 'babbage-search-query', 'ada-similarity', 'gpt-3.5-turbo', 'whisper-1', 'text-search-ada-doc-001', 'text-search-babbage-query-001', 'code-search-ada-code-001', 'curie-search-document', 'text-search-davinci-query-001', 'text-search-curie-doc-001', 'gpt-3.5-turbo-0301', 'babbage-search-document', 'babbage-code-search-text', 'davinci-instruct-beta', 'davinci-search-query', 'text-similarity-babbage-001', 'text-davinci-002', 'code-search-babbage-text-001', 'babbage', 'text-search-davinci-doc-001', 'code-search-ada-text-001', 'ada-search-query', 'text-similarity-ada-001', 'gpt-4', 'ada-code-search-code', 'ada', 'text-davinci-edit-001', 'davin

In [7]:
def push_move(board: chess.Board, move: chess.Move):
    """ 
    args:
        board (chess.Board)
        moves (chess.Move)
    """
    if board.is_legal(move):
        board.push(move)
    return board


def is_legal_move(board: chess.Board, move: chess.Move) -> bool:
    """ 
    Function that given a fen an a move checks
    if the provided move is legal
    args:
        board (chess.Board)
        moves (str)
    returns:
        bool  
    """
    return board.is_legal(move)

In [8]:
def build_filename(template, temperature, dir, model, ext="txt"):
    """
        generate filename name, if it exists, it appends an index number
        at the end of it.
    """
    current_date = datetime.datetime.now().strftime("%d_%m")
    filename = f"{current_date}-{template}-{model}-{temperature}.{ext}"
    full_path = os.path.join(dir, filename)

    index = 1
    while os.path.exists(full_path):
        new_filename = f"{current_date}-{template}-{model}-{temperature}_{index}.{ext}"
        full_path = os.path.join(dir, new_filename)
        index += 1

    return full_path


def build_columns(variability=3):
    columns = []

    columns.append(f"puzzleId")
    columns.append(f"datasetMove")

    for i in range(variability):
        columns.append(f"gpt{i}-move")
        columns.append(f"gpt{i}-valid")
        columns.append(f"gpt{i}-done")

    columns.append(f"gpt-results")
    return columns

In [9]:
# build dataframe that with the provided moves, the puzzle is completed
# should be a redundant code as the dataset should be all puzzles completed
# but just in case of some puzzle stored not done correctly.

checkmate_df = pd.DataFrame(columns=df.columns)

for index, row in df_one.iterrows():
    board = chess.Board(row['FEN'])
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]

    for move in chess_moves:
        if (move in board.legal_moves):
            push_move(board=board, move=move)

    if (board.is_checkmate()):
        checkmate_df = pd.concat([checkmate_df, row.to_frame().T])
    else:
        print(row['PuzzleId'])

In [21]:
# global constant variables

# output dir to save data files
OUTPUT_DIR = "./out"
# name of files to append to filename
TEMPLATE_FILE_PATH = "data"
# type of extension for data files
EXTENSION_FILE = "csv"

# error files dir path
OUTPUT_ERROR_DIR = "./out/error"
# name of error files to append to error filename
TEMPLATE_ERROR_FILE_PATH = "error"

# openai variables

TEMPERATURE = 1
# AVAILABLE MODELS [gpt-3.5-turbo-16k, gpt-4, gpt-4-0613]
MODEL = 'gpt-3.5-turbo-16k-0613'

# config variables

# increase time in case of error *`ServiceUnavailableError: The server is overloaded or not ready yet.`*
REQ_INTERVAL = 5  # 10 sec
VARIABILITY_TRIES = 3
MAX_PUZZLES = 50

# debug flag
DEBUG = True

In [11]:
def request(
    prompt="",
    context=np.array([]),
):
    """
    Helper function to make a request to openai api, and return it's response.
    args:
        prompt (string)
        model (str)
        temperature (int)
    returns:
        str
    """
    context = np.append(
        context,
        np.array([{
            "role": 'user',
            "content": f"""
                {prompt}
            """,
        }]), axis=0)

    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=context.tolist(),
        temperature=TEMPERATURE,  # this is the degree of randomness of the model's output
        max_tokens=10,
    )

    return response.choices[0].message["content"]

In [24]:
def process_test_row(row, context, temperature=0, debug=False):
    # create chess board with 'fen' dataset
    board = chess.Board(row['FEN'])
    # create chess moves from 'uci' moves from dataset
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]

    # retrieve last move
    last_move = chess_moves.pop()

    # add all moves except the last one.
    for move in chess_moves:
        push_move(board=board, move=move)

    # generate prompt
    prompt = f"""
        "fen": {board.fen()}
        "valid_moves": { json.dumps([move.uci() for move in list(board.legal_moves)])}
        "history": {json.dumps([move.uci() for move in list(chess_moves)])}
    """

    # req to gpt model
    req = request(prompt=prompt, context=context).replace(" ", "")

    if (debug):
        print(f"[gpt-response]: {req}")

    res = literal_eval(req)
    # check if valid gpt res move
    valid_res = is_legal_move(
        board=board, move=chess.Move.from_uci(res['move']))

    # info printings
    if (debug):
        print(f"[gpt-move | dataset-move] : {res['move']} | {last_move}")

    if (valid_res is not True):
        return {
            "completed": False,
            "move": res['move'],
            "valid": valid_res
        }

    board_gpt = push_move(board=copy.deepcopy(
        board), move=chess.Move.from_uci(res['move']))

    return {
        "completed": board_gpt.is_checkmate(),
        "move": res['move'],
        "valid": valid_res
    }

In [16]:
context = np.array([])

# for training-dataset purposes
# for index, row in train_df_one.iterrows():
#     context = np.append(context, process_training_row(row))

messages = np.array([
    {
        "role": "system",
        "content": f"""      
                You are a chess engine that solves chess puzzles. 
                Your objective is to do mate in one in the puzzle I provide you.
                                    
                The prompt input structure will be something like this delimited by the three backticks:
                                    
                 ```
                "fen": <fen>
                "valid_moves" : <valid_moves>
                "history": <history>
                ```

                "<fen>" stands for chess board position in 'FEN (Forsyth–Edwards Notation)' format.
                "<valid_moves>" stands for all legal moves available for that FEN board in UCI (Universal \Chess Interface) format.
                "<history>" stands for all the moves that has been played in that puzzle n UCI (Universal \Chess Interface) format.
                                    
                Your task is to pick a move from the "valid_moves" list above
                that maximizes your chance of doing checkmate.
                                                    
                Output the best move in UCI format to follow this position. Use the following single blob of JSON. Do not include any other information.
            
                ```
                {{"move": <generated_move>}}
                ```
            """,
    },
])

context = np.append(messages, context)

In [25]:
filename = build_filename(
    template=TEMPLATE_FILE_PATH,
    temperature=TEMPERATURE,
    dir=OUTPUT_DIR,
    model=MODEL,
    ext=EXTENSION_FILE
)

error_filename = build_filename(
    template=TEMPLATE_ERROR_FILE_PATH,
    temperature=TEMPERATURE,
    dir=OUTPUT_ERROR_DIR,
    model=MODEL,
)

print(f"[exp-path] : {filename}")
print(f"[error-exp-path] : {error_filename}")

_df = pd.DataFrame(columns=build_columns(
    VARIABILITY_TRIES)).reset_index(drop=True)

try:
    FILE = open(filename, 'w')
    for index, row in checkmate_df.iterrows():
        try:
            # only first 100 rows [testing]
            if (index == MAX_PUZZLES):
                break

            # list to store results
            results = []
            final_column = []
            # store puzzle id
            results.append(row['PuzzleId'])
            # store dataset last move
            results.append(row['Moves'].split().pop())

            # 3 iterations for variability purposes
            for i in range(VARIABILITY_TRIES):
                try:
                    result = process_test_row(
                        row=row, context=context, debug=False)
                    results.append(result['move'])
                    results.append(result['valid'])
                    results.append(result['completed'])
                    final_column.append(result['completed'])
                except Exception as e:
                    results.append(None)
                    results.append(False)
                    results.append(False)
                    final_column.append(False)

                if ("gpt-4" in MODEL):
                    time.sleep(REQ_INTERVAL)

            results.append(' '.join(str(e) for e in final_column))

            if (DEBUG is True):
                print(f"[{row['PuzzleId']}]: { results }")

            _df.loc[len(_df)] = results
        except Exception as e:
            if (DEBUG is True):
                print(f"[{row['PuzzleId']}]: { e }")

            ERROR_FILE = open(error_filename, 'w')
            ERROR_FILE.write(f"[{row['PuzzleId']}]: { e }\n")
            ERROR_FILE.close()

    print(f"Finished experiment and stored at {filename}")
except Exception as e:
    if (DEBUG is True):
        print(f"{ e }\n")

_df.to_csv(filename, index=False)


[exp-path] : ./out/08_08-data-gpt-3.5-turbo-16k-0613-1_1.csv
[error-exp-path] : ./out/error/08_08-error-gpt-3.5-turbo-16k-0613-1_6.txt
[001gi]: ['001gi', 'a5c3', 'h8g8', True, False, 'h8g8', True, False, 'd7e8', True, False, 'False False False']
[001wb]: ['001wb', 'g6h5', 'e8g8', True, False, 'e8g8', True, False, 'e8f8', True, False, 'False False False']
[002CP]: ['002CP', 'g6c2', 'g6h6', True, False, 'g6h8', False, False, 'g6h8', False, False, 'False False False']
[003IX]: ['003IX', 'e4g3', 'g4g8', True, False, 'e7e8', True, False, 'g4g1', True, False, 'False False False']
[004iZ]: ['004iZ', 'g3g7', 'd4g7', True, False, 'd4g7', True, False, 'd4g7', True, False, 'False False False']
[004zI]: ['004zI', 'h6h8', 'h6h8', True, True, 'h6h8', True, True, 'h6h8', True, True, 'True True True']
[008Nz]: ['008Nz', 'd1d8', 'd1d8', True, True, 'a5b6', True, False, 'h4h5', True, False, 'True False False']
[008o6]: ['008o6', 'a8f8', 'f1f8', True, True, 'f1f8', True, True, 'a8f8', True, True, 'True T